# 네이버 영화 댓글 감정 분석과 예측

캐글에서 실행 필수
https://www.kaggle.com/code/isc0304/naver-movie-comment-evaluation-and-prediction/notebook

## 필수 라이브러리 설치

In [4]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev -y
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5389 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6751 B]       
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [1578 B]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [361 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2313 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1712 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [27.5 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [966 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB] 
Get:13 htt

## 네이버 영화 사이트에서 리뷰 수집

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

review_page = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=184516&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}'
dict_score = dict()

for page_number in range(1, (7394//10) + 2):
    time.sleep(2)
    print(f'[+]와칸다 포에버 {page_number}페이지 조회 시작...')
    res = requests.get(review_page.format(page_number))
    bs4obj = BeautifulSoup(res.text, 'html.parser')

    for index, score_result in enumerate(bs4obj.find_all('div', {"class": 'score_result'})[0].find_all('li')):
        list0 = score_result
        score = int(list0.em.text) # 점수
        text = list0.find_all('span', {'id': f'_filtered_ment_{index}'})[0].text.strip()
        dict_score[f'{page_number}-{index}'] = [score,text]
    
    if page_number == 10:
        break

print(f'[+]와칸다 포에버 {page_number}페이지까지 조회 완료')    
pd.DataFrame(dict_score, index=['score','text']).T

[+]와칸다 포에버 1페이지 조회 시작...
[+]와칸다 포에버 2페이지 조회 시작...
[+]와칸다 포에버 3페이지 조회 시작...
[+]와칸다 포에버 4페이지 조회 시작...
[+]와칸다 포에버 5페이지 조회 시작...
[+]와칸다 포에버 6페이지 조회 시작...
[+]와칸다 포에버 7페이지 조회 시작...
[+]와칸다 포에버 8페이지 조회 시작...
[+]와칸다 포에버 9페이지 조회 시작...
[+]와칸다 포에버 10페이지 조회 시작...
[+]와칸다 포에버 10페이지까지 조회 완료


,score,text
1-0,1,아니 왜 0점은못주는거야
1-1,6,이게 맞는진 진짜 모르겠고. 채드윅보스만 배우님 그립습니다.
1-2,10,영화보면서 무한~도전~! 외치고 싶은걸 참았다
1-3,4,볼거리 정말 빈약하다. 아이언맨 디자인은 무슨 후레쉬맨 보는줄.
1-4,2,마블도 끝났구나....
...,...,...
10-5,4,2시간 40분의 채드윅 보스만 추모영화일뿐 음악이 좋아 그나마 버텼다
10-6,5,"블랙 팬서를 재밌게 봤기에, 많은 분들이 우려했던 긴 러닝타임도 전혀 문제가 되지 ..."
10-7,7,생각보다 잔잔.. 뭔가 우와 와 이런게 전혀없음ㅋㅋ
10-8,1,"솔직히 지금껏 본 마블 영화중에서는 제일 아쉬웠고 실망이 크네요. 각본, 대사, 스..."


## 깃헙 저장소로부터 리뷰 데이터 다운로드

In [6]:
# 리눅스 명령어를 사용해 다운로드 수행
!wget https://github.com/e9t/nsmc/raw/master/ratings_train.txt 

review_df = pd.read_csv('ratings_train.txt', sep='\t')
review_df.head()

--2022-11-21 15:04:21--  https://github.com/e9t/nsmc/raw/master/ratings_train.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt [following]
--2022-11-21 15:04:22--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.1s    

2022-11-21 15:04:22 (128 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]



,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
review_df.shape

(150000, 3)

## 데이터 전처리

In [8]:
from sklearn.model_selection import train_test_split

review_df.dropna(inplace=True)
x = review_df['document'].values
y = review_df['label'].values

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=0)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(119996,)
(29999,)
(119996,)
(29999,)


In [9]:
from konlpy.tag import Twitter
twitter = Twitter()
twitter.morphs(train_x[0])

/opt/conda/lib/python3.7/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['공감',
 '도',
 '안되고',
 '맘',
 '에',
 '드는',
 '구석',
 '이',
 '없다',
 '.',
 '극장',
 '에서',
 '이',
 '걸',
 '봤으니',
 '...']

TfidfVectorizer 기능은 텍스트를 벡터라이징하는 기능이다. 
- `tokenizer=twitter.morphs`는 형태소 분석기를 사용해 텍스틑 자르겠다는 의미
- `ngram_range=(1,2)`은 단어를 여러개 묶어서 벡터라이징을 수행해 순서를 인지할 수 있는 장점이 있다.
- `min_df=3` 데이터가 최소 3회이상 등장해야 특징으로 사용할 수 있다. (너무 희소한 것은 특징으로 사용하기 어렵다)
- `max_df=0.9` 데이터의 빈도수가 top 10% 들어가면 특징으로 사용할 수 없다. (너무 흔한 것은 특징으로 보기 어렵다)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Twitter

tfv = TfidfVectorizer(tokenizer=twitter.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x) # 텍스트를 숫자로 변환한 train_x가 완성된다.
tfv_train_x

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


<119996x107710 sparse matrix of type '<class 'numpy.float64'>'
	with 2438600 stored elements in Compressed Sparse Row format>

## 평점 학습 수행

In [12]:
from sklearn.linear_model import LogisticRegression # 이진분류 알고리즘
from sklearn.model_selection import GridSearchCV    # 하이퍼파리미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [13]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'C': 3}
0.8621954065135504


## 평점 예측 수행

In [ ]:
tfv_test_x = tfv.transform(test_x)

In [15]:
grid_cv.best_estimator_.score(tfv_test_x, test_y)

0.8623954131804393

In [17]:
a = ['아 너무 재미있어요 꼭 보세요',
     '핵노잼 너무 재미없어 절대 보지마',
     '영화 너무 훌륭해요!',
     '영화가 지루해요',
     '보지마삼'
    ]

my_review = tfv.transform(a)
grid_cv.best_estimator_.predict(my_review)

array([1, 0, 1, 0, 0])